In [54]:
import pandas as pd

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [56]:
#Load the dataset
df=pd.read_csv('Churn_modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [57]:
#Preprocess the data, drop irrelevant columns 
df=df.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [58]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [59]:
#encode categorical variables
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

In [60]:
#Geography wont be encoded: use one hot encoding

In [61]:
#Encode Geography using one hot encoding
from sklearn.preprocessing import OneHotEncoder

In [62]:
onehot_encoder=OneHotEncoder()

In [63]:
geo_encoded=onehot_encoder.fit_transform(df[['Geography']]).toarray()

In [64]:
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [65]:
onehot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [66]:
geo_encoded_df=pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))

In [67]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [68]:
#combine these columns with priginal data 
df=pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [69]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [70]:
#save these 
with open("label_encoder_gender.pkl", 'wb') as f:
    pickle.dump(label_encoder_gender, f)

In [71]:
with open("onehot_encoder_geography.pkl", 'wb') as f:
    pickle.dump(onehot_encoder, f)

In [72]:
#divide dataset into features and labels
X=df.drop('Exited', axis=1)
y=df['Exited']

In [ ]:
#split data into train and test sets 
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [74]:
#scale the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [75]:
with open("scaler.pkl", 'wb') as f:
    pickle.dump(scaler, f )

In [76]:
#tensorflow: Sequential model (ANN)
import tensorflow as tf

In [77]:
import tensorflow.keras

In [78]:
import sys
print(sys.executable)


c:\Users\nadee\Documents\ann\tfenv\python.exe


In [79]:
import keras 

In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [81]:
X_train.shape[1]

12

In [82]:
#Build ANN model
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[(1)],)),
    Dense(32, activation='relu'), 
    Dense(1, activation='sigmoid') #output layer
]) 

In [83]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [84]:
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
opt
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [85]:
#Compile model 
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [86]:
#Set up the tensorboard callback 
log_dir="logs/fit_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [87]:
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)


In [88]:
#set up early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [89]:
#Train the model 
history=model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100, 
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3898 - accuracy: 0.8361 - val_loss: 0.3716 - val_accuracy: 0.8525
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3554 - accuracy: 0.8571 - val_loss: 0.3508 - val_accuracy: 0.8615
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3478 - accuracy: 0.8605 - val_loss: 0.3562 - val_accuracy: 0.8565
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3444 - accuracy: 0.8593 - val_loss: 0.3362 - val_accuracy: 0.8645
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3409 - accuracy: 0.8589 - val_loss: 0.3376 - val_accuracy: 0.8575
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3358 - accuracy: 0.8620 - val_loss: 0.3443 - val_accuracy: 0.8580
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3355 - accuracy: 0.8605 - val_loss: 0.3507 - val_accuracy: 0.8695

In [90]:
model.save('model.h5')

c:\Users\nadee\Documents\ann\tfenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [91]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [92]:
import sys
print(sys.executable)


c:\Users\nadee\Documents\ann\tfenv\python.exe


In [93]:
%tensorboard --logdir logs/fit_20251021-185624

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
C:\Users\nadee\Documents\ann\tfenv\lib\site-packages\tensorboard\default.py:30: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-10-22 13:48:09.970455: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

Traceback (most recent call last):
  File "C:\Users\nadee\Documents\ann\tfenv\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\nadee\Documents\ann\tfenv\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File

In [94]:
#Load the pickle file
import tensorflow as tf
from tensorflow.keras.models import load_model 
import pickle
import pandas as pd 
import numpy as np

In [95]:
#Load the trained model, scaler pickle and onehot
model=load_model('model.h5')

#load the encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender=pickle.load(f)

with open('onehot_encoder_geography.pkl', 'rb') as f:
    label_encoder_geography=pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler=pickle.load(f)


In [96]:
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender': 'Male', 
    'Age':40,
    'Tenure':3,     
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [107]:
#one hot encode geography
geo_encoded=onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\nadee\Documents\ann\tfenv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [108]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [114]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [117]:
#Scaling the input data 
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [118]:
#Predict churn 
prediction= model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 89ms/step


array([[0.04480631]], dtype=float32)

In [119]:
prediction_probability=prediction[0][0]
prediction_probability

0.044806313

In [120]:
if prediction_probability>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
